In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Sep 23 14:40:53 2024

@author: ethankim
"""
import math
import random
import time
import numpy


AllowableWhiteSquare=[0,7,8,9,10,11,12]
AllowableBlackSquare=[0,1,2,3,4,5,6]
Decoder = [" ", chr(9817), chr(9814), chr(9816), chr(9815), chr(9813), chr(9812),chr(9823),chr(9820),chr(9822), chr(9821),chr(9819),chr(9818)]

#display_position is meant to display a board state and state who is to move.
#It is primarily meant to be used for debugging and demonstrations.
#It does not distinguish between "legal" positions and "illegal" positions
#Either if someone inputs the wrong code for a piece or the position is not
#possible in a real game.


def display_position(pos):
    x=math.sqrt(len(pos)-1)

    if x!=x//1 or x==0:

        print("Invalid Position")

    else:
       i=0
       while i<x:
           s=""
           j=0
           while j<x:
               s = s + str(Decoder[pos[int(j+i*x)]]) + " "
               j = j + 1
           print(s)
           i = i + 1
       if(pos[int(x**2)]==1):
           print("White to Move")
       elif(pos[int(x**2)]==2):
           print("Black to Move")

#All legal move functions requires us to know where the piece is.


def find_piece(pos,piece):
    y=[]
    index = 0
    for x in pos:
        if(x==piece):
            if(index!=len(pos)-1):
                y.append(index)
        index = index + 1
    return y


#Returns a list of all moves the rooke can make, ignoring check. a represents
#where the rooke is. It is made like this since now we can call upon this for
#the queen as well

def legal_rooke(pos, a):
    row_length=math.sqrt(len(pos)-1)
    row_length=int(row_length)
    pos_pos=[]
    if(pos[len(pos)-1]==1):
        b = -1
        Squares = AllowableWhiteSquare
    else:
        b = 1
        Squares = AllowableBlackSquare
    if(len(a)!=0):
        for i in a:

            c = pos[i]

            x = i

            #The theory behind this is that the while loop will stop iterating
            #if the piece they are about to hit is white. If white then all
            #takable pieces are greater than 6. Otherwise, all takable pieces
            #are less than 6.

            while x - row_length > -1 and pos[x-row_length] in Squares:
                x = x - row_length
                temp = pos.copy()
                temp[x] = c
                temp[i] = 0
                temp[len(pos)-1] = temp[len(pos)-1] - b
                pos_pos.append(temp.copy())

            #We also want to stop if the piece runs into a black piece

                if(pos[x]!=0):
                    break

            x = i

            while x + row_length < len(pos)-1 and (pos[x+row_length] in Squares):
                x = x + row_length
                temp = pos.copy()
                temp[x] = c
                temp[i] = 0
                temp[len(pos)-1] = temp[len(pos)-1] - b
                pos_pos.append(temp.copy())
                if(pos[x]!=0):
                    break

            x = i

            #Now going left. Mod is used out of convienence.

            while (x - 1) % row_length !=3 and (pos[x-1] in Squares):
                x = x - 1
                temp = pos.copy()
                temp[x] = c
                temp[i] = 0
                temp[len(pos)-1] = temp[len(pos)-1] - b
                pos_pos.append(temp.copy())
                if(pos[x]!=0):
                    break

            x = i

            #Now going right

            while (x + 1) % row_length != 0 and (pos[x+1] in Squares):
                x = x + 1
                temp = pos.copy()
                temp[x] = c
                temp[i] = 0
                temp[len(pos)-1] = temp[len(pos)-1] - b
                pos_pos.append(temp.copy())
                if(pos[x]!=0):
                    break


            #Now everything has been moved and castling is a king move so we
            #can return pos_pos

    return pos_pos


def legal_bishop(pos, a):
    row_length=math.sqrt(len(pos)-1)
    row_length=int(row_length)
    pos_pos=[]
    if(pos[len(pos)-1]==1):
        b = -1
        Squares = AllowableWhiteSquare
    else:
        b = 1
        Squares = AllowableBlackSquare
    for i in a:
        c = pos[i]

        x = i

        while(x - row_length - 1 > -1 and (x - row_length - 1) % row_length != 3) and (pos[x - row_length - 1] in Squares):
            x = x - row_length - 1
            temp = pos.copy()
            temp[x] = c
            temp[i] = 0
            temp[len(pos)-1] = temp[len(pos)-1] - b
            pos_pos.append(temp.copy())
            if(pos[x]!=0):
                break

        x = i

        while(x - row_length + 1 > -1 and (x - row_length + 1) % row_length != 0) and (pos[x - row_length + 1] in Squares):
            x = x - row_length + 1
            temp = pos.copy()
            temp[x] = c
            temp[i] = 0
            temp[len(pos)-1] = temp[len(pos)-1] - b
            pos_pos.append(temp.copy())
            if(pos[x]!=0):
                break

        x = i
        while(x + row_length + 1 < len(pos)-1 and (x + row_length + 1) % row_length != 0) and (pos[x + row_length + 1] in Squares):
            x = x + row_length + 1
            temp = pos.copy()
            temp[x] = c
            temp[i] = 0
            temp[len(pos)-1] = temp[len(pos)-1] - b
            pos_pos.append(temp.copy())
            if(pos[x]!=0):
                break


        x = i

        while(x + row_length - 1 < len(pos)-1 and (x + row_length - 1) % row_length != 3) and (pos[x + row_length - 1] in Squares):
            x = x + row_length - 1
            temp = pos.copy()
            temp[x] = c
            temp[i] = 0
            temp[len(pos)-1] = temp[len(pos)-1] - b
            pos_pos.append(temp.copy())
            if(pos[x]!=0):
                break

    return pos_pos

#Now we can just declate the queen is both of the previous two functions
#combined which will be done in the legal_moves function for consistency

#The Knight shall be defined more step by step then previous ones. Because of
#the weird way it moves, we check if we're out of bounds one step at a time
#rather than all at once.

def legal_knight(pos,a):
    row_length=math.sqrt(len(pos)-1)
    row_length=int(row_length)
    pos_pos=[]
    if(pos[len(pos)-1]==1):
        Squares = AllowableWhiteSquare
        b = -1
    else:
        b = 1
        Squares = AllowableBlackSquare
    for i in a:
        if(i - row_length >= 0):
           if((i-row_length-1) % row_length != 3 and (i-row_length-1) % row_length != 0 and (pos[i - row_length - 2] in Squares)):
               temp = pos.copy()
               temp[i] = 0
               temp[i-row_length-2] = pos[i]
               temp[len(pos)-1] = temp[len(pos)-1] - b
               pos_pos.append(temp.copy())
           if((i-row_length+1) % row_length != 0 and (i-row_length+1) % row_length != 3 and (pos[i - row_length + 2] in Squares)):
               temp = pos.copy()
               temp[i] = 0
               temp[i-row_length+2] = pos[i]
               temp[len(pos)-1] = temp[len(pos)-1] - b
               pos_pos.append(temp.copy())
           if(i - 2 * row_length >=0):
               if((i-2*row_length-1) % row_length != 3 and (pos[i - 2*row_length - 1] in Squares)):
                   temp = pos.copy()
                   temp[i] = 0
                   temp[i-2*row_length-1] = pos[i]
                   temp[len(pos)-1] = temp[len(pos)-1] - b
                   pos_pos.append(temp.copy())
               if((i-2*row_length+1) % row_length != 0 and (pos[i - 2*row_length + 1] in Squares)):
                   temp = pos.copy()
                   temp[i] = 0
                   temp[i-2*row_length+1] = pos[i]
                   temp[len(pos)-1] = temp[len(pos)-1] - b
                   pos_pos.append(temp.copy())
        if(i + row_length < len(pos)-1):
            if((i+row_length-1) % row_length != 3 and (i+row_length-1) % row_length != 0 and (pos[i + row_length - 2] in Squares)):
                temp = pos.copy()
                temp[i] = 0
                temp[i+row_length-2] = pos[i]
                temp[len(pos)-1] = temp[len(pos)-1] - b
                pos_pos.append(temp.copy())
            if((i+row_length+1) % row_length != 0 and (i+row_length+1) % row_length != 3 and (pos[i + row_length + 2] in Squares)):
                temp = pos.copy()
                temp[i] = 0
                temp[i+row_length+2] = pos[i]
                temp[len(pos)-1] = temp[len(pos)-1] - b
                pos_pos.append(temp.copy())
            if(i + 2 * row_length <len(pos)-1):
                if((i+2*row_length-1) % row_length != 3 and (pos[i + 2*row_length - 1] in Squares)):
                    temp = pos.copy()
                    temp[i] = 0
                    temp[i+2*row_length-1] = pos[i]
                    temp[len(pos)-1] = temp[len(pos)-1] - b
                    pos_pos.append(temp.copy())
                if((i+2*row_length+1) % row_length != 0 and (pos[i + 2*row_length + 1] in Squares)):
                    temp = pos.copy()
                    temp[i] = 0
                    temp[i+2*row_length+1] = pos[i]
                    temp[len(pos)-1] = temp[len(pos)-1] - b
                    pos_pos.append(temp.copy())
    return pos_pos

#Now pawns

def legal_pawn(pos,a):
    row_length=math.sqrt(len(pos)-1)
    row_length=int(row_length)
    pos_pos=[]
    if(pos[len(pos)-1]==1):
        b = -1
        Squares = AllowableBlackSquare
    else:
        b = 1
        Squares = AllowableWhiteSquare
    for i in a:
        x = i
        if(x + b * row_length > -1 and x + b * row_length < len(pos) - 1 and pos[x + b * row_length]==0):
            x = x+b*row_length
            temp = pos.copy()
            temp[i] = 0
            temp[x] = pos[i]
            temp[len(pos)-1] = temp[len(pos)-1] - b
            if(b == -1):
                z=[]
                y = 0
                while(y<row_length):
                    z.append(0+y)
                    y = y + 1
                if(x in z):
                    j = 1
                    while (j < 5):
                        temp[x] = 1 + j
                        pos_pos.append(temp.copy())
                        j = j + 1
                else:
                    pos_pos.append(temp.copy())
            else:
                z=[]
                y = 0
                while(y<row_length):
                    z.append(len(pos)-2-y)
                    y = y + 1
                if(x in z):
                    j = 1
                    while (j < 5):
                        temp[x] = 7 + j
                        pos_pos.append(temp.copy())
                        j = j + 1
                else:
                    pos_pos.append(temp.copy())
        if((i + b * row_length - 1) % row_length != 3 and (pos[i + b * row_length-1] not in Squares)):
            x = i + b * row_length - 1
            temp = pos.copy()
            temp[i] = 0
            temp[i+b * row_length-1] = pos[i]
            temp[len(pos)-1] = temp[len(pos)-1] - b
            if(b == -1):
                z=[]
                y = 0
                while(y<row_length):
                    z.append(0+y)
                    y = y + 1
                if(x in z):
                    j = 1
                    while (j < 5):
                        temp[i+b * row_length - 1] = 1 + j
                        pos_pos.append(temp.copy())
                        j = j + 1
                else:
                    pos_pos.append(temp.copy())
            else:
                z=[]
                y = 0
                while(y<row_length):
                    z.append(len(pos)-2-y)
                    y = y + 1
                if(x in z):
                    j = 1
                    while (j < 5):
                        temp[i+b * row_length - 1] = 7 + j
                        pos_pos.append(temp.copy())
                        j = j + 1
                else:
                    pos_pos.append(temp.copy())
        if((i + b * row_length + 1) % row_length != 0 and (pos[i + b * row_length+1] not in Squares)):
            x = i + b * row_length + 1
            temp = pos.copy()
            temp[i] = 0
            temp[i+b * row_length+1] = pos[i]
            temp[len(pos)-1] = temp[len(pos)-1] - b
            if(b == -1):
                z=[]
                y = 0
                while(y<row_length):
                    z.append(0+y)
                    y = y + 1
                if(x in z):
                    j = 1
                    while (j < 5):
                        temp[i+b * row_length+1] = 1 + j
                        pos_pos.append(temp.copy())
                        j = j + 1
                else:
                    pos_pos.append(temp.copy())
            else:
                z=[]
                y = 0
                while(y<row_length):
                    z.append(len(pos)-2-y)
                    y = y + 1
                if(x in z):
                    j = 1
                    while (j < 5):
                        temp[i+b * row_length+1] = 7 + j
                        pos_pos.append(temp.copy())
                        j = j + 1
                else:
                    pos_pos.append(temp.copy())
    return pos_pos

#Finally, the king

def legal_king(pos, a):
    row_length=math.sqrt(len(pos)-1)
    row_length=int(row_length)
    pos_pos=[]
    if(pos[len(pos)-1]==1):
        b = -1
        Squares = AllowableWhiteSquare
    else:
        b = 1
        Squares = AllowableBlackSquare
    for i in a:
        if((i+1) % row_length != 0 and (pos[i+1] in Squares)):
            temp = pos.copy()
            temp[i] = 0
            temp[i+1] = pos[i]
            temp[len(pos)-1] = temp[len(pos)-1] - b
            pos_pos.append(temp.copy())
        if((i-1) % row_length != 3 and (pos[i-1] in Squares)):
            temp = pos.copy()
            temp[i] = 0
            temp[i-1] = pos[i]
            temp[len(pos)-1] = temp[len(pos)-1] - b
            pos_pos.append(temp.copy())
        if((i-row_length) > -1 and (pos[i-row_length] in Squares)):
            temp = pos.copy()
            temp[i] = 0
            temp[i-row_length] = pos[i]
            temp[len(pos)-1] = temp[len(pos)-1] - b
            pos_pos.append(temp.copy())
        if((i - row_length - 1) > -1 and (i - row_length - 1) % row_length != 3 and (pos[i-row_length - 1] in Squares)):
            temp = pos.copy()
            temp[i] = 0
            temp[i-row_length - 1] = pos[i]
            temp[len(pos)-1] = temp[len(pos)-1] - b
            pos_pos.append(temp.copy())
        if((i - row_length + 1) > -1 and (i - row_length + 1) % row_length != 0 and (pos[i-row_length + 1] in Squares)):
            temp = pos.copy()
            temp[i] = 0
            temp[i-row_length +1] = pos[i]
            temp[len(pos)-1] = temp[len(pos)-1] - b
            pos_pos.append(temp.copy())
        if((i+row_length) < len(pos) - 1 and (pos[i+row_length]) in Squares):
            temp = pos.copy()
            temp[i] = 0
            temp[i+row_length] = pos[i]
            temp[len(pos)-1] = temp[len(pos)-1] - b
            pos_pos.append(temp.copy())
        if((i + row_length - 1) < len(pos) - 1 and (i + row_length - 1) % row_length != 3 and (pos[i+row_length - 1] in Squares)):
            temp = pos.copy()
            temp[i] = 0
            temp[i+row_length - 1] = pos[i]
            temp[len(pos)-1] = temp[len(pos)-1] - b
            pos_pos.append(temp.copy())
        if((i + row_length + 1) < len(pos) - 1 and (i + row_length + 1) % row_length != 0 and (pos[i+row_length + 1] in Squares)):
            temp = pos.copy()
            temp[i] = 0
            temp[i+row_length +1] = pos[i]
            temp[len(pos)-1] = temp[len(pos)-1] - b
            pos_pos.append(temp.copy())
    return pos_pos

#Now we create a function to discover all of these moves

def legal_moves(pos):
    pos_pos=[]
    b = pos[len(pos)-1] - 1
    a = find_piece(pos, 1 + b * 6)
    pos_pos.append(legal_pawn(pos, a).copy())
    a = find_piece(pos, 2 + b * 6)
    pos_pos.append(legal_rooke(pos, a).copy())
    a = find_piece(pos, 3 + b * 6)
    pos_pos.append(legal_knight(pos, a).copy())
    a = find_piece(pos, 4 + b * 6)
    pos_pos.append(legal_bishop(pos, a).copy())
    a = find_piece(pos, 5 + b * 6)
    #We summon all queen moves here for consistency
    pos_pos.append(legal_rooke(pos, a).copy())
    pos_pos.append(legal_bishop(pos, a).copy())
    a = find_piece(pos, 6 + b * 6)
    pos_pos.append(legal_king(pos, a).copy())
    return pos_pos


#I was trying to avoid this, but I suppose I can fix it by adjusting legal
#moves later. For now, it determines if a player has moved into check (or
#failed to counter check) by determing the legal moves of the other player
#and seeing if any of them involve the king disappearing. It's ine-eficient,
#but I'll see what I can do about that later

def is_check(pos,player):
    pos_copy=pos.copy()
    if(player == 1):
        if(len(find_piece(pos, 6))==0):
            return True
        pos_copy[len(pos_copy)-1] = 2
        toBeTested = legal_moves(pos_copy)
        for a in toBeTested:
            for moves in a:
                if(len(find_piece(moves, 6))==0):
                    return True
        return False
    else:
        if(len(find_piece(pos, 12))==0):
            return True
        pos_copy[len(pos_copy)-1] = 1
        toBeTested = legal_moves(pos_copy)
        for a in toBeTested:
            for moves in a:
                if(len(find_piece(moves, 12))==0):
                    return True
        return False


#Quick function to remove checks

def remove_check(legal_moves):
    #Michael Strauss helped me with this for loop
    for piece in legal_moves:
        piece_copy = piece[:]
        for move in piece_copy:
            if(move[len(move)-1]==1):
                b = 1
                c = 2
            else:
                b = 2
                c = 1
            if(is_check(move, c)):
                piece.remove(move)




#Now the big boy. For now, a basic code will be presented to run one game.
#A more detailed thing is to come, but I am tired


'''

Initial_Position = [ 8, 9, 10, 12,7,7,7,7,1,1,1,1,2,3,4,6,1]

pos = Initial_Position

amount = 1

count = 0

Stalemate = False

while(amount > 0):
    display_position(pos)
    a = legal_moves(pos[:]).copy()

    remove_check(a)

    amount = 0

    for i in a:
            amount = len(i) + amount

    if(amount != 0):
        index = random.randint(1, amount)

        i = 0

        j = 0

        while(len(a[j]) + i < index):
            i = len(a[j]) + i
            j = j+1

        index = index - i

        prev_pos = pos.copy()


        pos = a[j][index-1]

        if(numpy.sum(prev_pos)==numpy.sum(pos)+1 or numpy.sum(prev_pos)==numpy.sum(pos)-1 or numpy.sum(prev_pos)==numpy.sum(pos)):
            count = count + 1
        else:
            count = 0

    if(count > 49):
        print("Stalemate")
        Stalemate = True
        print(count)
        break




if(not Stalemate):
    if(is_check(pos, 1)):
       print("Black Wins")
    elif (is_check(pos,2)):
       print("White Wins")
    else:
       print("Stalemate")


'''













'\n\nInitial_Position = [ 8, 9, 10, 12,7,7,7,7,1,1,1,1,2,3,4,6,1]\n\npos = Initial_Position\n\namount = 1\n\ncount = 0\n\nStalemate = False\n\nwhile(amount > 0):\n    display_position(pos)\n    a = legal_moves(pos[:]).copy()\n\n    remove_check(a)\n\n    amount = 0\n\n    for i in a:\n            amount = len(i) + amount\n\n    if(amount != 0):\n        index = random.randint(1, amount)\n\n        i = 0\n\n        j = 0\n\n        while(len(a[j]) + i < index):\n            i = len(a[j]) + i\n            j = j+1\n\n        index = index - i\n\n        prev_pos = pos.copy()\n\n\n        pos = a[j][index-1]\n\n        if(numpy.sum(prev_pos)==numpy.sum(pos)+1 or numpy.sum(prev_pos)==numpy.sum(pos)-1 or numpy.sum(prev_pos)==numpy.sum(pos)):\n            count = count + 1\n        else:\n            count = 0\n\n    if(count > 49):\n        print("Stalemate")\n        Stalemate = True\n        print(count)\n        break\n\n\n\n\nif(not Stalemate):\n    if(is_check(pos, 1)):\n       print("B

In [ ]:
'''
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Oct 10 15:58:03 2024

@author: ethankim
"""
#from chess import legal_moves
#from chess import remove_check
import os

position_list=[]
pos=[0,0,0,0,7,7,7,7,1,1,1,1,0,0,0,0,1]
white = [6,5,4,3,2]
black = [12,11,10,9,8]



for x in black:
    pos[1] = 0
    pos[2] = 0
    pos[3] = 0
    pos[0] = x
    for y in black:
        pos[1] = 0
        pos[2] = 0
        pos[3] = 0
        if((y == 12 and pos.count(12) < 1) or (y == 11 and pos.count(11)<1) or (y != 12 and y != 11 and pos.count(y)<2)):
            pos[1] = y
            for z in black:
                pos[2] = 0
                pos[3] = 0
                if(((z == 11 and pos.count(11)<1) or (z != 12 and z != 11 and pos.count(z)<2)) and pos.count(12)==1):
                    pos[2] = z
                    for a in black:
                        pos[3] = 0
                        if(((a == 11 and pos.count(11)<1) or (a != 12 and a != 11 and pos.count(a)<2)) and pos.count(12)==1):
                            pos[3] = a
                            for i in white:
                                pos[13] = 0
                                pos[14] = 0
                                pos[15] = 0
                                pos[12] = i
                                for j in white:
                                    pos[13] = 0
                                    pos[14] = 0
                                    pos[15] = 0
                                    if((j == 6 and pos.count(6) < 1) or (j == 5 and pos.count(5)<1) or (j != 6 and j != 5 and pos.count(j)<2)):
                                        pos[13] = j
                                        for k in white:
                                            pos[14] = 0
                                            pos[15] = 0
                                            if(((k == 6 and pos.count(6) < 1) or (k == 5 and pos.count(5)<1) or (k != 6 and k != 5 and pos.count(k)<2))):
                                                pos[14] = k
                                                for l in white:
                                                    pos[15] = 0
                                                    if(((l == 6 and pos.count(6) < 1) or (l == 5 and pos.count(5)<1) or (l != 6 and l != 5 and pos.count(l)<2)) and (pos.count(6)==1 or l == 6)):
                                                        pos[15] = l
                                                        position_list.append(pos.copy())

openings = open("Openings.txt", 'w')

for pos_temp in position_list:
    print(pos_temp, file = openings)

openings.close()
'''


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Oct 14 11:40:32 2024

@author: ethankim
"""
#from chess import legal_moves
#from chess import remove_check
#from chess import is_check
#from chess import display_position
import os

def straightener(vector_of_vectors):
    vector=[]
    for bob in vector_of_vectors:
        for vec in bob:
            vector.append(vec)
    return vector


def string_to_int_int_array(string):
    string = string.strip()

    string = string.split(']')

    vector = []

    string[0]=string[0][1:]

    string = string[:len(string)-2]

    i = 0

    for s in string:
        string_copy = string[i]
        string_copy = string_copy.split(',')
        if(i == 0):
            string_copy[0] = int(string_copy[0][1:])
        else:
            string_copy = string_copy[1:]
            string_copy[0] = int(string_copy[0][2:])
        j = 0
        for object in string_copy:
            if(j>0):
                string_copy[j] = int(string_copy[j])
            j = j + 1
        vector.append(string_copy)
        i = i + 1
    return vector

#Plan: For each pos in positions this cuntion creates a file listing the possible
#moves. If that file already exits, then it will instead read that file. This
#function will then go recursively. If all solutions are found to be bad from
#a certainn point it will return that result. Otherwise, it will burn access to
#those branches and return a benificial or alright result. We'll mark these results
#in a vector in the return statement. Hopefully will work? I wonder what this
#is called.

def solver(positions, maximum = 0):
    print_copy = positions[:]
    player = positions[0][16]
    index = 0
    vector = []
    Stalemate_count = 0
    for pos in positions:
        if(os.path.isfile("/content/moves/" + str(pos)+".txt")):
            done = open("/content/moves/" + str(pos)+".txt",'r')
            for line in done:
                if(line == "Stalemate\n"):
                    Stalemate_count = Stalemate_count + 1
                    Stalemate_pos = index
                elif(line == "Black Wins\n"):
                    return [0,0,1,False]
                elif(line == "White Wins\n"):
                    return [1,0,0,False]
                else:
                    line = string_to_int_int_array(line)
                    vector.append(line)
            done.close()
        else:
            record = open("/content/moves/" + str(pos)+".txt",'w')
            temp = legal_moves(pos)
            remove_check(temp)
            amount = 0
            for j in temp:
                    amount = len(j) + amount
            if(amount==0):
                if(is_check(pos, 1)):
                   print("Black Wins", file = record)
                   copy = positions[:]
                   for i in range(0,index):
                       positions.remove(copy[i])
                   for i in range(index+1, len(copy)):
                       positions.remove(copy[i])
                   return([0,0,1,False])
                elif (is_check(pos,2)):
                   print("White Wins", file = record)
                   copy = positions[:]
                   for i in range(0,index):
                       positions.remove(copy[i])
                   for i in range(index+1, len(copy)):
                       positions.remove(copy[i])
                   return([1,0,0,False])
                else:
                    print("Stalemate", file = record)
                    Stalemate_count = Stalemate_count + 1
                    Stalemate_pos = index
            else:
                temp = straightener(temp)
                print(temp, file = record)
                vector.append(temp)
            record.close()
            index = index + 1
    if(maximum<=0):
        return([0,0,0,True])
    Loss_count = 0
    k = 0
    for pos in vector:
        a = solver(pos,maximum - 1)
        record = open("/content/moves/" + str(positions[k])+".txt", 'w')
        print(pos, file = record)
        record.close()
        if(player == 1 and a[2] > 0):
            copy = positions[:]
            for i in range(0,k):
                positions.remove(copy[i])
            for i in range(k+1, len(copy)):
                positions.remove(copy[i])
            return([0,0,1,False])
        elif(player == 2 and a[0] > 0):
            copy = positions[:]
            for i in range(0,k):
                positions.remove(copy[i])
            for i in range(k+1, len(copy)):
                positions.remove(copy[i])
            return([1,0,0,False])
        elif(a[0] > 0 or a[2]>0):
            Loss_count = Loss_count + 1
        elif(a[1] > 0):
            Stalemate_count = Stalemate_count + 1
            Stalemate_pos = k
        k = k + 1
    if(Stalemate_count + Loss_count == len(positions)):
        if(Stalemate_count > 0):
            copy = positions[:]
            for i in range(0,Stalemate_pos):
                positions.remove(copy[i])
            for i in range(Stalemate_pos+1, len(copy)):
                positions.remove(copy[i])
            return([0,1,0,False])
        else:
            if(player == 1):
                return([1,0,0,False])
            else:
                return([0,0,1,False])
    else:
        return([0,0,0,True])






In [ ]:
def string_to_int_array(s):
    s=s.strip()
    s=s.split(',')
    Array=[]
    Array.append(int(s[0][1:]))
    for i in range(1,len(s)-1):
        Array.append(int(s[i]))
    Array.append(int(s[len(s)-1][1:2]))
    return Array

Openings_file = open("Openings.txt", 'r')
Openings = []
for line in Openings_file:
    Openings.append(string_to_int_array(line))

Openings_file.close()
len(Openings)

9840

In [ ]:
Openings_file = open("White_Wins_in_one.txt", 'r')
count = 0
for line in Openings_file:
    count = count + 1
Openings_file.close()
count

7687

In [ ]:
import os
import shutil

if os.path.exists("/content/moves/"):
    shutil.rmtree("/content/moves/")
os.mkdir("/content/moves/")

In [ ]:
Openings = [[8, 12,11,8,7,7,7,7,1,1,1,1,2,6,5,2,1]]
len(Openings)
print(Openings)

[[8, 12, 11, 8, 7, 7, 7, 7, 1, 1, 1, 1, 2, 6, 5, 2, 1]]


In [ ]:
'''
#from BasicSolver import solver
def string_to_int_array(s):
    s=s.strip()
    s=s.split(',')
    Array=[]
    Array.append(int(s[0][1:]))
    for i in range(1,len(s)-1):
        Array.append(int(s[i]))
    Array.append(int(s[len(s)-1][1:2]))
    return Array


Unsolved = []
Solved_White_Wins = []
Solved_Black_Wins = []
Solved_Stalemate = []

for opening in Openings:
    temp = [opening]
    solver(temp,1)
    solver(temp,2)
    solver(temp,3)
    solver(temp,4)
    a = solver(temp,7)
    if(a[3]):
        Unsolved.append(opening)
    elif(a[0]==1):
        Solved_White_Wins.append(opening)
    elif(a[1]==1):
        Solved_Stalemate.append(opening)
    else:
        Solved_Black_Wins.append(opening)


Openings_file = open("Openings.txt", 'w')
for opening in Unsolved:
    print(opening, file = Openings_file)
Openings_file.close()

Openings_file = open("White_Wins_in_six.txt", 'w')
for opening in Solved_White_Wins:
    print(opening, file = Openings_file)
Openings_file.close()

Openings_file = open("Black_Wins_in_six1.txt", 'w')
for opening in Solved_Black_Wins:
    print(opening, file = Openings_file)
Openings_file.close()

Openings_file = open("Stalemate_in_six2.txt", 'w')
for opening in Solved_Stalemate:
    print(opening, file = Openings_file)
Openings_file.close()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
'''
import os
#import shutil
#shutil.rmtree('/content/drive/My Drive/[10,12,10,11,7,7,7,7,1,1,1,1,6,4,3,5,1]')
os.mkdir('/content/drive/My Drive/[8, 12, 11, 8, 7, 7, 7, 7, 1, 1, 1, 1, 2, 6, 5, 2, 1]')

In [ ]:
'''
import shutil
shutil.move('/content/moves', '/content/drive/My Drive/[8, 12, 11, 8, 7, 7, 7, 7, 1, 1, 1, 1, 2, 6, 5, 2, 1]')

'/content/drive/My Drive/[8, 12, 11, 8, 7, 7, 7, 7, 1, 1, 1, 1, 2, 6, 5, 2, 1]/moves'

To get the test cases, start with opening 1 in openings.txt exists before merge. We'll choose a variety of unsolved things and hope we don't get unlucky and find one that is solved in 5 and not too many are found. \\
First position: \\
[10, 12, 10, 11, 7, 7, 7, 7, 1, 1, 1, 1, 6, 4, 3, 5, 1] \\
solved in seven  \\
Second Position: \\
[9, 12, 9, 8, 7, 7, 7, 7, 1, 1, 1, 1, 3, 6, 4, 5, 1] \\
solved in nine \\
Third Position: \\
[8, 12, 8, 9, 7, 7, 7, 7, 1, 1, 1, 1, 2, 2, 4, 6, 1] \\
solved in seven \\
Fourth Position: \\
[12, 11, 10, 10, 7, 7, 7, 7, 1, 1, 1, 1, 6, 5, 4, 4, 1] \\
solved in nine \\
Fifth Position: \\
[12, 11, 9, 8, 7, 7, 7, 7, 1, 1, 1, 1, 5, 6, 4, 3, 1] \\
solved in nine \\
Sixth Position: \\
[12, 11, 8, 9, 7, 7, 7, 7, 1, 1, 1, 1, 3, 2, 6, 5, 1] \\
solved in seven \\
Seventh Position: \\
[12, 10, 10, 11, 7, 7, 7, 7, 1, 1, 1, 1, 6, 4, 2, 2, 1] \\
solved in seven \\
Eighth Position: \\
[12, 9, 10, 9, 7, 7, 7, 7, 1, 1, 1, 1, 4, 6, 5, 4, 1] \\
solved in nine \\
Ninth Position: \\
[12, 8, 8, 11, 7, 7, 7, 7, 1, 1, 1, 1, 6, 3, 5, 3, 1] \\
solved in nine \\
Tenth Position: \\
[8, 12, 11, 8, 7, 7, 7, 7, 1, 1, 1, 1, 2, 6, 5, 2, 1] \\
solved in seven \\
FROM WIKIPEDIA


In [ ]:
'''
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Oct 14 11:40:32 2024

@author: ethankim
"""
#from chess import legal_moves
#from chess import remove_check
#from chess import is_check
#from chess import display_position
import os



#Plan: For each pos in positions this cuntion creates a file listing the possible
#moves. If that file already exits, then it will instead read that file. This
#function will then go recursively. If all solutions are found to be bad from
#a certainn point it will return that result. Otherwise, it will burn access to
#those branches and return a benificial or alright result. We'll mark these results
#in a vector in the return statement. Hopefully will work? I wonder what this
#is called.

def memoryless_solver(positions, maximum = 0):
    print_copy = positions[:]
    player = positions[0][16]
    index = 0
    vector = []
    Stalemate_count = 0
    for pos in positions:
        if(os.path.isfile("/content/moves/" + str(pos)+".txt")):
            done = open("/content/moves/" + str(pos)+".txt",'r')
            for line in done:
                if(line == "Stalemate\n"):
                    Stalemate_count = Stalemate_count + 1
                    Stalemate_pos = index
                elif(line == "Black Wins\n"):
                    return [0,0,1,False]
                elif(line == "White Wins\n"):
                    return [1,0,0,False]
                else:
                    line = string_to_int_int_array(line)
                    vector.append(line)
            done.close()
        else:
            record = open("/content/moves/" + str(pos)+".txt",'w')
            temp = legal_moves(pos)
            remove_check(temp)
            amount = 0
            for j in temp:
                    amount = len(j) + amount
            if(amount==0):
                if(is_check(pos, 1)):
                   print("Black Wins", file = record)
                   copy = positions[:]
                   for i in range(0,index):
                       positions.remove(copy[i])
                   for i in range(index+1, len(copy)):
                       positions.remove(copy[i])
                   return([0,0,1,False])
                elif (is_check(pos,2)):
                   print("White Wins", file = record)
                   copy = positions[:]
                   for i in range(0,index):
                       positions.remove(copy[i])
                   for i in range(index+1, len(copy)):
                       positions.remove(copy[i])
                   return([1,0,0,False])
                else:
                    print("Stalemate", file = record)
                    Stalemate_count = Stalemate_count + 1
                    Stalemate_pos = index
            else:
                temp = straightener(temp)
                print(temp, file = record)
                vector.append(temp)
            record.close()
            index = index + 1
    if(maximum<=0):
        return([0,0,0,True])
    Loss_count = 0
    k = 0
    for pos in vector:
        a = memoryless_solver(pos,maximum - 1)
        record = open("/content/moves/" + str(positions[k])+".txt", 'w')
        print(pos, file = record)
        record.close()
        if(player == 1 and a[2] > 0):
            copy = positions[:]
            for i in range(0,k):
                positions.remove(copy[i])
            for i in range(k+1, len(copy)):
                positions.remove(copy[i])
            return([0,0,1,False])
        elif(player == 2 and a[0] > 0):
            copy = positions[:]
            for i in range(0,k):
                positions.remove(copy[i])
            for i in range(k+1, len(copy)):
                positions.remove(copy[i])
            return([1,0,0,False])
        elif(a[0] > 0 or a[2]>0):
            Loss_count = Loss_count + 1
        elif(a[1] > 0):
            Stalemate_count = Stalemate_count + 1
            Stalemate_pos = k
        k = k + 1
    if(Stalemate_count + Loss_count == len(positions)):
        if(Stalemate_count > 0):
            copy = positions[:]
            for i in range(0,Stalemate_pos):
                positions.remove(copy[i])
            for i in range(Stalemate_pos+1, len(copy)):
                positions.remove(copy[i])
            return([0,1,0,False])
        else:
            if(player == 1):
                return([1,0,0,False])
            else:
                return([0,0,1,False])
    else:
        return([0,0,0,True])






In [ ]:
'''
#from BasicSolver import memoryless_solver
import shutil
def string_to_int_array(s):
    s=s.strip()
    s=s.split(',')
    Array=[]
    Array.append(int(s[0][1:]))
    for i in range(1,len(s)-1):
        Array.append(int(s[i]))
    Array.append(int(s[len(s)-1][1:2]))
    return Array

Unsolved = []
Solved_White_Wins = []
Solved_Black_Wins = []
Solved_Stalemate = []

for opening in Openings:
    temp = [opening]
    a = memoryless_solver(temp,4)
    if(a[3]):
        Unsolved.append(opening)
    elif(a[0]==1):
        Solved_White_Wins.append(opening)
    elif(a[1]==1):
        Solved_Stalemate.append(opening)
    else:
        Solved_Black_Wins.append(opening)
    shutil.rmtree('/content/moves/')
    os.mkdir("/content/moves/")


Openings_file = open("/content/drive/My Drive/Openings.txt", 'w')
for opening in Unsolved:
    print(opening, file = Openings_file)
Openings_file.close()

Openings_file = open("/content/drive/My Drive/White_Wins_in_four.txt", 'w')
for opening in Solved_White_Wins:
    print(opening, file = Openings_file)
Openings_file.close()

Openings_file = open("/content/drive/My Drive/Black_Wins_in_four.txt", 'w')
for opening in Solved_Black_Wins:
    print(opening, file = Openings_file)
Openings_file.close()

Openings_file = open("/content/drive/My Drive/Stalemate_in_four.txt", 'w')
for opening in Solved_Stalemate:
    print(opening, file = Openings_file)
Openings_file.close()

First round, 280 completed. Fixed problem so hopefully will manage to run the rest.

Note to self. Might have to restart it eventually since made an error in solver. Error is fine, just have to check there weren't any stalemates in the early stages at some point.